예를 들어 GUI 애플리케이션 에서 문서 저장 위치를 선택할 수 있는 대화창을 띄우고 싶다

대화창이 표시하는 정보는 이벤트 핸들러의 인자를 통해 구체적으로 전달된다.

하지만 대화창이 사용자 선호 설정 등과 같은 전역 상태를 읽어야 자기 자신을 화면에 그릴 수 있다

`dialog.py`


문제는 prefs 객체가 들어있는 app 모듈이 프로그램 시작시 대화창을 표시하고자 앞에서 정의한 dialog 클래스를 임포트 한다는 점이다.

이로 인해 순환 의존 관계가 생겼다.

실행하면 에러 발생

파이썬의 임포트 기능이 일반적으로 어떻게 작동하는지 알아야한다.

모듈이 임포트되면 파이썬이 실제로 어떤일을 하는지 우선순위로 나열

1. sys.path에서 모듈 위치를 검색한다.
2. 모듈의 코드를 로딩하고 컴파일되는지 확인한다.
3. 임포트할 모듈에 상응하는 빈 모듈 객체를 만든다.
4. 모듈을 sys.modules에 넣는다.
5. 모듈 객체에 있는 코드를 실행해서 모듈의 내용을 정의한다

순환 의존관계에서 문제는 어떤 모듈의 애트리뷰트를 정의하는 코드 (5단계)가 실제로 실행되기 전까지는

모듈 애트리뷰트가 정의되지 않는다는 점이다.

하지만 모듈 자체는 import 문을 사용해서 sys.modules에 추가되자마자 import문을 사용해 로드할 수 있다.

In [ ]:
위 예제에서 app 모듈은 다른 모든 내용을 정의하기 전에 dialog 모듈을 임포트한다. 그 후 dialog 모듈은 app을 임포트 한다.

app이 아직 실행 (app은 지금 dialog를 임포트하는 중이다. ) 되지 않았기 때문에 app 모듈은 비어있다 (4단계)

따라서 prefs 애트리뷰트를 정의하는 코드가 아직 실행되지 못했기 때문에 (dialog의 5단계를 실행하는 도중)에 AttributeError가 발생한다.

In [ ]:
이 문제를 해결하는 가장 좋은 방법은 코드를 리팩터링해서 prefs 데이터 구조를 의존 관계트리의 맨 밑바닥으로 보내는 것이다

이렇게 변경하고

app과 dialog 모두 (prefs가 들어있는 )같은 유틸리티 모듈을 임포트하고 순환 임포트를 피할 수 있다.

하지만 리팩터링이 너무 어려워서 노력할만한 가치가 없거나 아예 이런 식의 명확한 구분이 불가능한 경우도 있다.

순환 임포트를 깨는 다른 세 가지 방법이 있다.

In [ ]:
1. 임포트 순서를 바꾸는 것이다. 예를 들어 app 모듈의 다른 내용이 모두 실행된 다음 맨 뒤에서 dialog 모듈을 임포트하면 AttributeError가 사라진다.
    `88_ex2`
    
    이런 코드가 제대로 작동하는 이유는 dialog 모듈이 나중에 로딩 될 때 dialog 안에서 재귀적으로 임포트한 app에 app.prefs가 이미 정의 돼 있기 때문이다.
    (app에 대해 5단계가 거의 다 수행됨)


    이런 방식이 AttributeError를 없애주기는 하지만 PEP 8  스타일 가이드에 위배 된다

    항상 맨 위에 넣으라고 제안. 그래서 이방법은 권하지 않음

2. 임포트 시점에 부작용을 최소화 하는 모듈을 사용하는 것이다.
    
    `88_ex3`
    모듈이 함수 클래스 상수만 정의하게 하고 임포트 시점에 실제로 함수를 전혀 실행하지 않게 만듣나

    그 후 다른 모듈이 모두 임포트를 끝낸 후 호출 할 수 있는 configure 함수를 제공한다.

    configure 함수의 목적은 다른 모듈들의 애트리뷰트에 접근해 모듈 상태를 준비하는 것이다.

    다른 모듈의 (5단계)가 끝난 후 configure를 실행하므로 configure가 실행되는 시점에는 항상 모든 애트리뷰트가 정의 돼 있다.

    이런 구조는 대부분 잘 작동하며 의존 관계 주입 같은 다른 패턴을 적용할 수도 있다.

    모듈 안에 서로 다른 단계가 둘 이상 있으면 객체를 정의하는 부분과 객체를 설정하는 부분이 분리되기 떄문에 코드를 읽기가 더 어려워진다.

3. 동적 임포트

    프로그램이 실행 되는 동안 모듈 임포트가 일어나는 것

    `88_ex4`

    동적 임포트 방식은 임포트 방식을 구조적으로 바꾸지 않아도 된다는 점에서 장점이 있다.

    일반적으로 이런 동적 임포트는 피하면 좋으나 프로그램 전체 구조를 바꾸는 것보다 더 나은 경우가 많다.